In [2]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import mechanicalsoup

data_path = Path.cwd()

# Main URL: https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/?tablelist=true&rxid=bdf9d8da-96f1-4100-ae09-18cb3eaeb313
url = "https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/?tablelist=true&rxid=bdf9d8da-96f1-4100-ae09-18cb3eaeb313"
base_url = "https://openstat.psa.gov.ph"
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

def get_csv(url, filename):
  browser = mechanicalsoup.StatefulBrowser()
  browser.open(url)
  page = browser.page
  form = browser.select_form('form[method="post"]')

  # Get VARS and OPTIONS, 1:1 mapped
  varnames = [var.string for var in page.css.select("#VariableSelection")[0].find_all(class_="variableselector_valuesselect_variabletitle")]
  varnames.append("OutputFormats")
  options = []
  selects = page.css.select("#VariableSelection")[0].find_all("select")
  for e in selects:
    options.append([opt["value"] for opt in e.find_all("option")])

  options[2] = options[2][:11]  # hotfix to "Number of selected cells exceeds the maximum allowed 100,000"
  
  # Set OPTIONS for each VAR
  for i in range(0, len(selects)-1):
    browser[selects[i]["name"]] = options[i]
  browser[selects[-1]["name"]] = "FileTypeCsvWithHeadingAndComma"

  # Submit request
  response = browser.submit_selected()

  # Download CSV
  with open(str(data_path) + "/datasets/cropyield/%s.csv" % filename, "w+") as text_file:
      text_file.write(response.text.split("\n",2)[2]) # Remove first 2 lines from string

for child in soup.ol.children:
  if child != "\n" and child.a is not None:
    table_url = base_url + child.a["href"].split("?")[0][:-1]
    if table_url[-5] == "A": continue # avoid appendices
    filename = str(child.a.string.split(":")[0])
    filename = f"farmgate_{filename.lower().replace(' ', '')}"
    print(filename, table_url)
    get_csv(table_url, filename)


farmgate_palayandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0012E4EVCP0.px
farmgate_palayandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0022E4EAHC0.px
farmgate_riceandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0032E4ECNV0.px
farmgate_palay https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0042E4EPFU0.px
farmgate_corn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0052E4ECFU0.px
farmgate_othercrops https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0062E4EVCP1.px
farmgate_othercrops https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0072E4EAHO0.px
farmgate_othercrops https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0082E4ENBT0.px
farmgate_palayandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0092E4ENVCP.px
farmgate_palayandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0102E4ENAHC.px
farmgate_palayandcorn https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/011